# GPU gauge transformation

### Set parametres and eviroment variables

In [1]:
import numpy as np

# hbar * c [GeV * fm]
hbarc = 0.197326 

# Simulation box 
L = 2      
# N = 128   
N = 64 
tau_sim = 1     
DTS = 8     

# Glasma fields
su_group = 'su3'
Qs = 2        
ns = 50    
factor = 0.8        
g2mu = Qs / factor     
g = np.pi * np.sqrt(1 / np.log(Qs / 0.2))          		
mu = g2mu / g**2          	
ir = 0.1 * g**2 * mu         
uv = 10.0       

# TODO: Run more events
nevents = 1

In [2]:
import os

# I need to add this line to ask resources from a specific GPU, which is free. Our GPU server has no queing system
os.environ["CUDA_VISIBLE_DEVICES"]="3"

# os.environ["MY_NUMBA_TARGET"] = "numba"
os.environ["MY_NUMBA_TARGET"] = "cuda"
os.environ["PRECISION"] = "double"
os.environ['GAUGE_GROUP'] = su_group

# Import relevant modules
import sys
sys.path.append('..')

# Glasma modules
import curraun.core as core
import curraun.mv as mv
import curraun.initial as initial
initial.DEBUG = False

import curraun.su as su
from curraun.numba_target import use_cuda
if use_cuda:
    from numba import cuda

import curraun.su as su
import curraun.lc_gauge as lc_gauge

Using CUDA
Using SU(3)
Using double precision


### We define the simulation routine

In [3]:
# import pickle
from tqdm import tqdm

# Simulation routine
def simulate(): 
    # Derived parameters
    a = L / N
    E0 = N / L * hbarc
    DT = 1.0 / DTS
    maxt = int(tau_sim / a * DTS)

    # Initialize Glasma fields
    s = core.Simulation(N, DT, g)
    va = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
    vb = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
    initial.init(s, va, vb)

    # Initialize LC gauge transformation
    lc = lc_gauge.LCGaugeTransf(s)

    # Plus gauge link
    # uplus_lc = su.GROUP_TYPE(np.zeros((maxt//DTS-1, N*N, su.GROUP_ELEMENTS)))

    with tqdm(total=maxt) as pbar:
        for t in range(maxt):            
            # Evolve Glasma fields
            core.evolve_leapfrog(s)

            if t%DTS == 0:
                lc.initialize_vlc()

                # I checked that vlc0 is correctly initialized an swaped with vlc1
                # vlc0 = lc.vlc0.copy()
                # print('vlc0=', vlc0[0, :])
                # print('vlc0=',vlc0[7, :])

                for xplus in range(maxt//DTS):
                    lc.evolve_lc(xplus)
                    # if xplus > t:
                        # print(xplus)
                        # vlc0 = lc.vlc0.copy()
                        # print('vlc0=', vlc0[7, :])
                        # vlc1 = lc.vlc1.copy()
                        # print('vlc1=', vlc1[7, :])
                    # elif xplus == (t//DTS) and xplus != 0:
                    if xplus == (t//DTS) and xplus != 0:
                        print(xplus)
                        # uplus_lc = lc.up_lc.copy()
                        # print("uplus_lc=", uplus_lc[7, :])
                        uplus_temp = lc.up_temp.copy()
                        print("uplus_temp=", uplus_temp[7, :])


            pbar.update(1)

    if use_cuda:
        cuda.current_context().deallocations.clear()

    return uplus_lc

In [4]:
# Supress various horribly long warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
uplus_lc = simulate()